In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
np.random.seed(1)

卷积：
1. padding 
  np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), 'constant', constant_values = 0)
  参数含义：(padding谁， 沿那个轴padding、padding多少， padding方式， padding的值)

In [ ]:
def zero_pad(X,pad):
#     X_paded = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), 'constant', constant_values = 0)    
    X_paded = np.pad(X,(
                        (0,0),       #样本数，不填充
                        (pad,pad),   #图像高度,你可以视为上面填充x个，下面填充y个(x,y)
                        (pad,pad),   #图像宽度,你可以视为左边填充x个，右边填充y个(x,y)
                        (0,0)),      #通道数，不填充
                        'constant', constant_values=0)      #连续一样的值填充
    return X_paded

In [ ]:
np.random.seed(1)
x = np.random.randn(4,3,3,2)
x_paded = zero_pad(x,2)
#查看信息
print ("x.shape =", x.shape)
print ("x_paded.shape =", x_paded.shape)
print ("x[1, 1] =", x[1, 1])
print ("x_paded[1, 1] =", x_paded[1, 1])

#绘制图
fig , axarr = plt.subplots(1,2)  #一行两列
axarr[0].set_title('x')
axarr[0].imshow(x[0,:,:,0])
axarr[1].set_title('x_paded')
axarr[1].imshow(x_paded[0,:,:,0])

In [ ]:
def conv_single_step(a_slice_prev, W, b):  #  取出来的3维长方体和W的大小相同，做卷积，每个通道变成一个数，所有通道再相加，就是最后卷积值
    s = np.multiply(a_slice_prev, W) + b
    Z = np.sum(s)
    return Z
np.random.seed(1)

#这里切片大小和过滤器大小相同
a_slice_prev = np.random.randn(4,4,3)
W = np.random.randn(4,4,3)
b = np.random.randn(1,1,1)

Z = conv_single_step(a_slice_prev,W,b)

print("Z = " + str(Z))




In [ ]:
def conv_forward(A_prev, W, b, hparameters):
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    pad = hparameters["pad"]
    stride = hparameters["stride"]
    A_prev_pad = zero_pad(A_prev, pad)
    
    n_H = int((n_H_prev + 2 * pad - f) / stride) + 1
    n_W = int((n_W_prev + 2 * pad - f) / stride) + 1
    Z = np.zeros((m, n_H, n_W, n_C))
    for i in range(m):
        i_A_prev_pad = A_prev_pad[i] # 取出第i个padding的图像
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = stride * h
                    vert_end = vert_start + f
                    horiz_start = stride * w
                    horiz_end = horiz_start + f
                    a_slice_prev = i_A_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]  #取出小长条，所有通道的小图块堆叠成的
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[0, 0, 0, c])
    
    assert(Z.shape == (m, n_H, n_W, n_C))
    cache = (A_prev, W, b, hparameters)                
    return (Z , cache)

In [ ]:
np.random.seed(1)

A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)

hparameters = {"pad" : 2, "stride": 1}

Z , cache_conv = conv_forward(A_prev,W,b,hparameters)

print("np.mean(Z) = ", np.mean(Z))
print("cache_conv[0][1][2][3] =", cache_conv[0][1][2][3])

In [ ]:
def pool_forward(A_prev,hparameters,mode="max"):
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    f = hparameters["f"]
    stride = hparameters["stride"]
    n_H = int((n_H_prev - f) / stride) + 1
    n_W = int((n_W_prev - f) / stride) + 1
    n_C = n_C_prev  #池化层通道数不变，只是每个通道尺寸变小
    A = np.zeros((m, n_H, n_W, n_C))
    for i in range(m):
        i_image = A_prev[i,:,:,:] #取出第i个
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    h_start = h * stride
                    h_end = h_start + f
                    w_start = w * stride
                    w_end = w_start + f
                    a_slice_prev = A_prev[i, h_start:h_end, w_start:w_end, c]  # 取出小长条,进行pooling操作
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_slice_prev)  #numpy基本函数，取最大值
                    if mode == "average":
                        A[i, h, w, c] = np.mean(a_slice_prev)
    assert(A.shape == (m, n_H, n_W, n_C)) 
    cache = (A_prev, hparameters)
    return A,cache

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2,4,4,3)
hparameters = {"f":4 , "stride":1}

A , cache = pool_forward(A_prev,hparameters,mode="max")
A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A =", A)
print("----------------------------")
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A =", A)

In [ ]:
# 卷积的反向传播原理

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as maimg
import tensorflow as tf
from tensorflow.python.framework import ops
import cnn_utils
%matplotlib inline
np.random.seed(1)

In [ ]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = cnn_utils.load_dataset()

In [ ]:
index = 6
plt.imshow(train_set_x_orig[index])
print ("y = " + str(np.squeeze(train_set_y_orig[:, index])))

In [63]:
X_train = train_set_x_orig / 255
X_test = test_set_x_orig / 255
Y_train = cnn_utils.convert_to_one_hot(train_set_y_orig, 6).T
Y_test = cnn_utils.convert_to_one_hot(test_set_y_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}


(1, 1080)
[5 0 2 ... 2 4 5]
Y.shape =  (6, 1080)
(1, 120)
[0 0 0 5 1 0 3 1 5 1 5 1 3 1 1 3 5 4 0 4 5 4 2 5 3 5 4 2 1 2 3 1 0 3 1 1 0
 4 2 3 0 3 0 2 3 1 2 2 0 3 4 1 2 0 4 0 4 0 4 4 5 5 2 4 4 5 0 1 3 5 0 4 1 2
 3 4 3 5 1 5 2 0 1 4 2 4 4 1 4 5 5 0 0 5 5 5 3 3 5 2 2 2 0 2 5 3 0 2 3 4 1
 3 2 4 2 2 1 3 1 3]
Y.shape =  (6, 120)
number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [64]:
# 创建placeholder
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0])  # 输入图片的数量不定，所以设置为None
    Y = tf.placeholder(tf.float32, [None, n_y])   # 矩阵的行数代表输入图片的数量
    return X,Y

In [65]:
X , Y = create_placeholders(64,64,3,6)
print ("X = " + str(X))
print ("Y = " + str(Y))

X = Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(?, 6), dtype=float32)


In [68]:
# 初始化参数
def initialize_parameters():
    tf.set_random_seed(1)
    w1 = tf.get_variable("W1", [4, 4, 3, 8], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    w2 = tf.get_variable("W2", [2, 2, 8, 16], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"W1":w1, "W2":w2}
    return parameters

In [69]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters()
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2 = " + str(parameters["W2"].eval()[1,1,1]))

    sess_test.close()

W1 = [ 0.00131723  0.1417614  -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
W2 = [-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]
